- Inspired from demo.ipynb in NGSIM.jl
- Takeaways
    - 101 has 5 segments
    - Start from bottom of screen, 1 is bottom of screen, 2 is exit ramp,
    3 is straight section, 4 is entry to road till entry ramp 
    join point, 5 is entry ramp

- For `observe!` from `Tim2DDriver` to work, had to make the following changes
    - Had to modify `AutomotiveDrivingModels.jl/src/2d/vehicles/scene_records.jl` to extend
the default length of the capacity in the record container from 100 to 300
    - Did the same capacity extension in `Records.jl/src/frames.jl`, `QueueRecords.jl` and 
`ListRecords.jl`
- Note that querying Tim2DDriver yields a `latLonAccel` return type. Had to create an array
to enable these values to work with `step`

In [1]:
using NGSIM
using AutomotiveDrivingModels
using AutoViz
using Interact # For slider bar trajectory propagation
using Reel

HTML{String}("    <script class='js-collapse-script'>\n        var curMatch =\n            window.location.href\n            .match(/(.*?)\\/notebooks\\/.*\\.ipynb/);\n\n        curMatch = curMatch ||\n            window.location.href\n            .match(/(.*?)\\/apps\\/.*\\.ipynb/);\n\n        if ( curMatch ) {\n            \$('head').append('<base href=\"' + curMatch[1] + '/\">');\n        }\n    </script>\n")

HTML{String}("<script class='js-collapse-script' src='/assetserver/5dc1e185f8b2175037a0bb4bac5baa1c8ca39ea5-assets/webio/dist/bundle.js'></script>")

HTML{String}("<script class='js-collapse-script' src='/assetserver/5dc1e185f8b2175037a0bb4bac5baa1c8ca39ea5-assets/providers/ijulia_setup.js'></script>")

HTML{String}("  <script class='js-collapse-script'>\n    \$('.js-collapse-script').parent('.output_subarea').css('padding', '0');\n  </script>\n")

In [2]:
roadway = ROADWAY_101;

In [39]:
using AutoEnvs

n_veh = 3
filepath = Pkg.dir("NGSIM", "data", 
    "trajdata_i101_trajectories-0805am-0820am.txt")
params = Dict(
        "trajectory_filepaths"=>[filepath],
        "H"=>200,
        "primesteps"=>50,
        "n_veh"=>n_veh,
        "remove_ngsim_veh"=>false
)
# env = MultiagentNGSIMEnvVideoMaker(params)
env = MultiagentNGSIMEnv(params);
timestep = 0.1;

In [ ]:
model = Tim2DDriver(timestep,mlane = MOBIL(timestep));

In [9]:
# Borrowed from `sisl/gail-driver/validation/validation.jl`
# Reduced T to 0.1 to see more aggressive driving
# Now wondering how to make the blue vehicle change lanes
mlon = IntelligentDriverModel(σ=0.1,k_spd=1.0,T=0.1,s_min=2.0,
    a_max=3.0,d_cmf=2.5)
mlat = ProportionalLaneTracker(σ=0.1, kp=3.0, kd=2.0)
mlane = MOBIL(timestep,politeness=0.01,advantage_threshold=0.01)

model = Tim2DDriver(timestep, mlon=mlon, mlat=mlat, mlane=mlane);

In [41]:
env.n_veh

3

In [42]:
# This will create a series of images in /tmp/episode1
# You can open the first and then press right arrow

# a = ones(n_veh, 2)
# a[2] = 0.0
reset(env,random_seed=9);
t_init = env.t

# @show env.n_veh
for _ in 1:200
    
    render(env)
    
    #------------------------------------------------
    a = zeros(env.n_veh,2)
#     @show a
    for (jj,veh) in enumerate(env.ego_vehs)
        observe!(model,env.scene,env.roadway,veh.id)
#         @show jj
        latlonacc = rand(model)
        
#         @show a[jj][1]
#         @show latlonacc.a_lat
        a[jj,1] = latlonacc.a_lat
#         @show "3"
        a[jj,2] = latlonacc.a_lon
    end
    
    
#     observe!(model, env.scene, env.roadway, env.ego_vehs[1].id)
#     latlonacc = rand(model)
#     a = [latlonacc.a_lon latlonacc.a_lat] # This form for AccelTurnRate
#     a = [latlonacc.a_lat latlonacc.a_lon]   # This form for LatLonAccel

        
    # Note: We have modified env to use LatLonAccel instead of AccelTurnRate
    x, r, terminal, _ = step(env, a)
end

In [ ]:
function makevid(t,dt)
    for _ in t:300
    render(env)
    observe!(model, env.scene, env.roadway, env.ego_vehs[1].id)
    latlonacc = rand(model)
    a = [latlonacc.a_lon latlonacc.a_lat]
    x, r, terminal, _ = step(env, a)
    end
end

In [ ]:
reset(env,random_seed=4)
film = roll(makevid, fps=10, duration=1.0)

In [ ]:
write("mygif.gif",film)

In [ ]:
# Take a look at sisl/gail-driver/julia/envs/Auto2D.jl function reel_drive
# for video making inspiration

## Video making using recorded cars on ngsim
- Trajdatas is what the car trajectories are stored in
- We want to color the ego vehicle differently to be able to see it

In [ ]:
# argument 1 loads i101 7:50 to 8:05.
# load_trajdata function defined in NGSIM.jl/src/trajdata.jl
td1 = load_trajdata(1); 

scene = Scene(500)
egoid = 546

# Drive here in the notebook. Replay the trajectory as recorded in the ngsim data
@manipulate for i in 1000:2000
    temp_scene = get!(scene,td1,i)
    
    carcolors = Dict{Int,Colorant}()
    for veh in temp_scene
        #@show veh.id
        # if veh id matches the egoid color it blue otherwise green
        carcolors[veh.id] = 
        in(veh.id, egoid) ? colorant"blue" : colorant"green"
    end
    render(temp_scene, ROADWAY_101, 
        cam=CarFollowCamera{Int}(546,5.0),
#         cam=StaticCamera(VecE2(1966400, 570900), 5.0),
#         cam=FitToContentCamera(0.),
        car_colors=carcolors)
end